In [2]:
from audiolm_pytorch import SoundStream, SoundStreamTrainer

soundstream = SoundStream(
    codebook_size = 1024,
    rq_num_quantizers = 8,
)

trainer = SoundStreamTrainer(
    soundstream,
    folder = '/home/ckwdani/Music/train-clean-100',
    batch_size = 4,
    grad_accum_every = 8,         # effective batch size of 32
    data_max_length = 320 * 32,
    num_train_steps = 100
).cuda()

trainer.train()



training with dataset of 27112 samples and validating with randomly splitted 1427 samples
0: soundstream total loss: 9.082, soundstream recon loss: 0.006 | discr (scale 1) loss: 2.000 | discr (scale 0.5) loss: 2.000 | discr (scale 0.25) loss: 2.000
0: saving to results
0: saving model to results
1: soundstream total loss: 10.150, soundstream recon loss: 0.006 | discr (scale 1) loss: 1.997 | discr (scale 0.5) loss: 1.999 | discr (scale 0.25) loss: 2.000
2: soundstream total loss: 11.648, soundstream recon loss: 0.007 | discr (scale 1) loss: 1.996 | discr (scale 0.5) loss: 1.999 | discr (scale 0.25) loss: 2.000
3: soundstream total loss: 10.630, soundstream recon loss: 0.004 | discr (scale 1) loss: 2.015 | discr (scale 0.5) loss: 1.998 | discr (scale 0.25) loss: 1.999
4: soundstream total loss: 17.088, soundstream recon loss: 0.005 | discr (scale 1) loss: 2.030 | discr (scale 0.5) loss: 2.000 | discr (scale 0.25) loss: 1.997
5: soundstream total loss: 9.976, soundstream recon loss: 0.005

In [1]:
from einops.layers import torch
from typing import List
from tensorflow import Tensor
from ctypes import Union
from audiolm_pytorch.data import SoundDataset
import torchaudio
from audiolm_pytorch import SoundStream, SoundStreamTrainer
from torch.utils.data import Dataset, DataLoader

soundstream = SoundStream(
    codebook_size = 1024,
    rq_num_quantizers = 8,
)

def curtail_to_shortest_collate(data):
    min_len = min(*[datum.shape[0] for datum in data])
    data = [datum[:min_len] for datum in data]
    return torch.stack(data)

test = torchaudio.load("/home/ckwdani/Music/train-clean-100/40/222/40-222-0000.flac")
ds = SoundDataset(
            "/home/ckwdani/Music/train-clean-100/40/222/",
            max_length = 320 * 32,
            target_sample_hz = soundstream.target_sample_hz,
            seq_len_multiple_of = soundstream.seq_len_multiple_of
        )
dltest = DataLoader(ds, collate_fn=curtail_to_shortest_collate, batch_size=1)
dltest.dataset[0].to("cuda")
soundstream.load("results/soundstream.0.pt")
testTensor = soundstream.encoder(test[0])

2023-01-02 13:32:07.413691: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-02 13:32:08.073308: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-02 13:32:08.073357: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-02 13:32:08.073361: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [7]:
from IPython.lib.display import Audio

Audio(soundstream.decoder(testTensor).detach(), rate=30000)

In [22]:
import sklearn.decomposition as sk
sk.PCA

['../tess/OAF_Sad/OAF_white_sad.wav',
 '../tess/OAF_happy/OAF_sour_happy.wav',
 '../tess/OAF_Sad/OAF_vine_sad.wav',
 '../tess/YAF_sad/YAF_room_sad.wav',
 '../tess/OAF_Fear/OAF_judge_fear.wav',
 '../tess/OAF_disgust/OAF_far_disgust.wav',
 '../tess/OAF_Pleasant_surprise/OAF_time_ps.wav',
 '../tess/YAF_fear/YAF_germ_fear.wav',
 '../tess/YAF_sad/YAF_tip_sad.wav',
 '../tess/YAF_fear/YAF_shack_fear.wav',
 '../tess/OAF_Fear/OAF_chief_fear.wav',
 '../tess/OAF_disgust/OAF_dog_disgust.wav',
 '../tess/OAF_Fear/OAF_chat_fear.wav',
 '../tess/OAF_happy/OAF_half_happy.wav',
 '../tess/OAF_angry/OAF_when_angry.wav',
 '../tess/YAF_sad/YAF_bar_sad.wav',
 '../tess/YAF_happy/YAF_south_happy.wav',
 '../tess/YAF_fear/YAF_dime_fear.wav',
 '../tess/OAF_happy/OAF_keg_happy.wav',
 '../tess/YAF_disgust/YAF_luck_disgust.wav',
 '../tess/YAF_sad/YAF_fail_sad.wav',
 '../tess/YAF_disgust/YAF_lease_disgust.wav',
 '../tess/OAF_Sad/OAF_pike_sad.wav',
 '../tess/OAF_happy/OAF_hire_happy.wav',
 '../tess/YAF_pleasant_surpris